<a href="https://colab.research.google.com/github/Siamak-salimy/NUS/blob/main/AI_Lab/D_tree/c.45.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np



In [2]:
# -------------------------------------------------
# 1. خواندن داده‌ها از GitHub (لینک raw)
# -------------------------------------------------
diabetes_url = "https://github.com/Siamak-salimy/NUS/raw/main/AI_Lab/D_tree/diabetes.xlsx"
newdata_url = "https://github.com/Siamak-salimy/NUS/raw/main/AI_Lab/D_tree/newdata.xlsx"

# بارگذاری داده‌های اصلی
df = pd.read_excel(diabetes_url)

# حذف سطر اول اضافه (اگر ستون 'sample' وجود داشت)
if 'sample' in df.columns:
    df = df.drop(columns=['sample'])

# تبدیل به عددی و پاک‌سازی
df = df.apply(pd.to_numeric, errors='coerce')
df = df.dropna(subset=['Outcome'])  # حذف سطرهایی که Outcome NaN است

# جایگزینی مقادیر 0 غیرمنطقی با NaN (در ویژگی‌های پزشکی)
zero_cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in zero_cols:
    if col in df.columns:
        df[col] = df[col].replace(0, np.nan)

df = df.dropna()  # حذف سطرهای دارای NaN

# جدا کردن ویژگی‌ها و هدف
X = df.drop('Outcome', axis=1)
y = df['Outcome'].astype(int)


In [3]:

# -------------------------------------------------
# 2. ساخت مدل درخت تصمیم (شبیه C5.0 با entropy)
# -------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = DecisionTreeClassifier(
    criterion='entropy',        # شبیه C4.5 / C5.0
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42
)

model.fit(X_train, y_train)

# ارزیابی
y_pred = model.predict(X_test)
print("دقت مدل:", accuracy_score(y_test, y_pred))
print("\nگزارش طبقه‌بندی:")
print(classification_report(y_test, y_pred))


دقت مدل: 0.7848101265822784

گزارش طبقه‌بندی:
              precision    recall  f1-score   support

           0       0.86      0.81      0.83        53
           1       0.66      0.73      0.69        26

    accuracy                           0.78        79
   macro avg       0.76      0.77      0.76        79
weighted avg       0.79      0.78      0.79        79



In [4]:

# -------------------------------------------------
# 3. نمایش ویژگی‌های مهم
# -------------------------------------------------
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nویژگی‌های مهم (بر اساس اهمیت):")
print(importance_df)



ویژگی‌های مهم (بر اساس اهمیت):
                    Feature  Importance
1                   Glucose    0.423139
7                       Age    0.222342
5                       BMI    0.179772
6  DiabetesPedigreeFunction    0.129153
4                   Insulin    0.029483
2             BloodPressure    0.016110
3             SkinThickness    0.000000
0               Pregnancies    0.000000


In [5]:

# -------------------------------------------------
# 4. پیش‌بینی برای newdata.xlsx
# -------------------------------------------------
try:
    new_data = pd.read_excel(newdata_url)
    if 'sample' in new_data.columns:
        new_data = new_data.drop(columns=['sample'])
    if 'Outcome' in new_data.columns:
        new_data = new_data.drop(columns=['Outcome'])

    # پیش‌بینی
    predictions = model.predict(new_data)
    probabilities = model.predict_proba(new_data)

    print("\n" + "="*50)
    print("پیش‌بینی برای داده‌های جدید (newdata.xlsx):")
    for i, pred in enumerate(predictions):
        prob = probabilities[i][1] if len(probabilities[i]) > 1 else 0
        status = "دیابت دارد" if pred == 1 else "دیابت ندارد"
        print(f"نمونه {i+1}: {status} (احتمال دیابت: {prob:.2%})")

except Exception as e:
    print("\n⚠️ فایل newdata.xlsx یافت نشد یا خطا دارد. در عوض، از ورودی کاربر استفاده می‌کنیم.")
    print("\nلطفاً مقادیر بیمار جدید را وارد کنید:")
    new_sample = []
    for col in X.columns:
        val = float(input(f"{col}: "))
        new_sample.append(val)

    new_sample = np.array(new_sample).reshape(1, -1)
    pred = model.predict(new_sample)[0]
    prob = model.predict_proba(new_sample)[0][1]

    if pred == 1:
        print(f"\n⚠️ تشخیص: بیمار احتمالاً دیابت دارد. (احتمال دیابت: {prob:.2%})")
    else:
        print(f"\n✅ تشخیص: بیمار احتمالاً دیابت ندارد. (احتمال عدم دیابت: {1-prob:.2%})")


پیش‌بینی برای داده‌های جدید (newdata.xlsx):
نمونه 1: دیابت ندارد (احتمال دیابت: 42.86%)
